# Nobel Model

Úlohy:
1. Losovat úmrtí
2. Losovat vakcinaci
3. Ty, kde vakcinace - úmrtí <= 14 dnů => nevakcinovat
4. S pravděpodobností P lidi z bodu 3. necháme zemřít


In [1]:
from tqdm.contrib.concurrent import process_map
from matplotlib import pyplot as plt
from common import *

import seaborn as sns
import pandas as pd
import numpy as np
import pickle
import os

In [2]:
sns.set_theme()

## Load data

In [3]:
df_deaths = pd.read_csv(DEATHS_FILE)
df_vacc = pd.read_csv(VACCINATION_FILE)
df_counts = pd.read_csv(COUNTS_FILE)

### Sanity checks

In [4]:
assert len(df_deaths) >= len(df_vacc.drop(columns="poradi_davky").groupby(["tyden", "vek"]).sum()), "Cropping applied?"
assert len(df_deaths["tyden"].unique()) == len(df_vacc["tyden"].unique()), "Weeks are not the same!"
assert len(df_deaths["vek"].unique()) == len(df_vacc["vek"].unique()) == len(df_counts["vek"].unique()), "Age groups are not the same!"

In [5]:
# We will only use first dose
df_vacc = df_vacc[df_vacc.poradi_davky == 1].drop(columns="poradi_davky")

# Convert weeks to timestamps in seconds
df_vacc["tyden"] = pd.to_datetime(df_vacc["tyden"] + "-1", format="%Y-W%W-%w").astype("int64") // 10**9
df_deaths["tyden"] = pd.to_datetime(df_deaths["tyden"] + "-1", format="%Y-W%W-%w").astype("int64") // 10**9

### Preview

In [6]:
df_deaths

,tyden,vek,umrti
0,1609113600,0-29,24
1,1609113600,30-39,27
2,1609113600,40-49,69
3,1609113600,50-59,168
4,1609113600,60-69,428
...,...,...,...
765,1674432000,30-39,15
766,1674432000,40-49,54
767,1674432000,50-59,133
768,1674432000,60-69,301


In [7]:
df_vacc

,tyden,vek,pocet_davek
0,1609113600,0-29,144
1,1609113600,30-39,233
2,1609113600,40-49,338
3,1609113600,50-59,274
4,1609113600,60-69,183
...,...,...,...
2293,1674432000,80-129,8
2309,1674432000,30-39,41
2312,1674432000,0-29,58
2314,1674432000,40-49,14


In [8]:
df_counts

,vek,celkem
0,0-29,3259827
1,30-39,1409650
2,40-49,1735533
3,50-59,1354501
4,60-69,1284689
5,70-79,1037997
6,80-129,441970


## Preparations

In [9]:
TIMEDELTA = timedelta(weeks=2).total_seconds()

In [10]:
def run(p):
    # Data variables
    people = {}
    
    # Variables for metrics
    count_vacc_after_death = 0
    count_vacc_timedelta_death = 0
    x, y11, y12, y21, y22 = {}, {}, {}, {}, {}
    
    # Generate vaccination and death date based on reality for reach person
    for row_counts in df_counts.itertuples():
        # Select rows for age group
        dfs_deaths_sel = df_deaths[df_deaths.vek == row_counts.vek]
        dfs_vacc_sel = df_vacc[df_vacc.vek == row_counts.vek]

        # Append None to the end of the array (= no death & no vaccination classes)
        da = np.append(dfs_deaths_sel.tyden, None)
        va = np.append(dfs_vacc_sel.tyden, None)

        # Append count to the end of the array for assignation with da and va (people with no death/vaccination)
        dp = np.append(dfs_deaths_sel.umrti, row_counts.celkem - dfs_deaths_sel.umrti.sum())
        vp = np.append(dfs_vacc_sel.pocet_davek, row_counts.celkem - dfs_vacc_sel.pocet_davek.sum())

        # Normalize (so that the sum is 1), then randomly choice dates of death/vaccination
        dates_death = np.random.choice(da, row_counts.celkem, p=dp / dp.sum())
        dates_vacc = np.random.choice(va, row_counts.celkem, p=vp / vp.sum())

        # print(f"Dead: {len(dates_death[dates_death != None])}/{dfs_deaths_sel.umrti.sum()}")

        # Assign dates for each person
        for n in range(row_counts.celkem):
            date_death = dates_death[n]
            date_vacc = dates_vacc[n]

            if date_vacc is not None and date_death is not None:
                # Invalid assigment
                if date_vacc > date_death:
                    count_vacc_after_death += 1
                    date_vacc = None

                # Modeled 
                elif date_vacc + TIMEDELTA > date_death:
                    if np.random.random() < p:
                        count_vacc_timedelta_death += 1
                        date_vacc = None

            people.setdefault(row_counts.vek, []).append({"dead": date_death, "vaccinated": date_vacc})

    # print(f"Vakcinováno po úmrtí: {count_vacc_after_death}")
    # print(f"Odebráno pro jev {p=}: {count_vacc_timedelta_death}");
    
    # Create report for graphs based on dates generated above
    for week in np.unique(df_deaths["tyden"]):
        for age, people_age in people.items():
            deaths_vacc = 0
            deaths_norm = 0
            alive_vacc = 0
            alive_norm = 0

            for i in people_age:
                # If selected person won't die in the simulation or die somewhere in the future
                if i["dead"] is None or i["dead"] > week:
                    if i["vaccinated"] is None or i["vaccinated"] > week:
                        alive_norm += 1
                    else:
                        alive_vacc += 1
                
                # If selected person is going to die this week
                elif i["dead"] == week:
                    # There is no need to check for bigger than death, it is filtered before
                    if i["vaccinated"] is None or i["vaccinated"] > week:
                        deaths_norm += 1
                    else:
                        deaths_vacc += 1

            # Save
            x.setdefault(age, []).append(datetime.fromtimestamp(week))
            y11.setdefault(age, []).append(deaths_vacc)
            y12.setdefault(age, []).append(alive_vacc)
            y21.setdefault(age, []).append(deaths_norm)
            y22.setdefault(age, []).append(alive_norm)
    
    return x, y11, y12, y21, y22, count_vacc_after_death, count_vacc_timedelta_death

## Simulation

In [48]:
P = 0
RUN_COUNT = 100

In [12]:
data = process_map(run, [P] * RUN_COUNT, max_workers=os.cpu_count())

  0%|          | 0/100 [00:00<?, ?it/s]

## Save data

In [17]:
with open(f"result/P_{P}_data.pickle", "wb") as f:
    pickle.dump(data2, f)

## Load data

In [49]:
with open(f"result/P_{P}_data.pickle", "rb") as f:
    data = pickle.load(f)

## Visualizations

In [50]:
COLORS = "bgrcmyk"

### Počet živých kumulativně

In [51]:
RESULTS_DIR = "result"

In [52]:
for directory, label in [
        ("alive_cumulative", "Počet živých lidí (kumulativně) dle vakcinance v čase"), 
        ("alive_derivative", "Počet živých lidí (derivace) dle vakcinance v čase"),
        ("dead", "Počet mrtvých lidí dle vakcinance v čase")
]:
    current_directory = os.path.join(RESULTS_DIR, directory)
    os.makedirs(current_directory, exist_ok=True)

    for age_low, age_up in AGE_CATEGORIES:
        age_category = f"{age_low}-{age_up}"

        fig, (al, ar) = plt.subplots(1, 2, sharey="row", figsize=(18, 6))
        was = set()

        for x, y11, y12, y21, y22, errored, removed in data:
            for age_n, age in enumerate(list(y12.keys())):
                if age == age_category:
                    if directory == "alive_derivative":
                        data_x = x[age][1:]
                        data_y1 = np.diff(y12[age])
                        data_y2 = np.diff(y22[age])
                        
                    elif directory == "alive_cumulative":
                        data_x = x[age]
                        data_y1 = y12[age]
                        data_y2 = y22[age]
                    
                    elif directory == "dead":
                        data_x = x[age]
                        data_y1 = y11[age]
                        data_y2 = y21[age]
                    else:
                        raise Exception("Something went really wrong.")
                        
                    al.plot(data_x, data_y1, color=COLORS[age_n], alpha=1 / RUN_COUNT, label=age if age not in was else None)
                    ar.plot(data_x, data_y2, color=COLORS[age_n], alpha=1 / RUN_COUNT, label=age if age not in was else None)
                    was.add(age)

        al.set_xlabel("Datum")
        ar.set_xlabel("Datum")

        al.set_ylabel(" ".join(label.split()[:2]))

        al.set_title("Vakcinovaní")
        ar.set_title("Nevakcinovaní")

        # al.legend()
        # ar.legend()

        fig.suptitle(label + f" pro {P=} a věkovou skupinu {age}")
        plt.savefig(os.path.join(current_directory, f"age={age_category}__{P=}.png"), facecolor="white", transparent=False)

        plt.clf()

/tmp/ipykernel_9841/255479627.py:12: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, (al, ar) = plt.subplots(1, 2, sharey="row", figsize=(18, 6))


<Figure size 1800x600 with 0 Axes>

<Figure size 1800x600 with 0 Axes>

<Figure size 1800x600 with 0 Axes>

<Figure size 1800x600 with 0 Axes>

...

<Figure size 1800x600 with 0 Axes>

<Figure size 1800x600 with 0 Axes>

<Figure size 1800x600 with 0 Axes>

<Figure size 1800x600 with 0 Axes>

<Figure size 1800x600 with 0 Axes>

### Účinnost

In [53]:
for directory, label in [
        ("efficacy", "Účinnost vakcíny")
]:
    current_directory = os.path.join(RESULTS_DIR, directory)
    os.makedirs(current_directory, exist_ok=True)

    for age_low, age_up in AGE_CATEGORIES:
        age_category = f"{age_low}-{age_up}"

        plt.figure(figsize=(16, 9))
        was = set()

        for x, y11, y12, y21, y22, errored, removed in data:
            for age_n, age in enumerate(list(y12.keys())):
                if age == age_category:
                    if directory == "efficacy":
                        data_x = x[age]
                        data_y = 1 - (np.array(y11[age]) / np.array(y12[age])) / (np.array(y21[age]) / np.array(y22[age]))
                        plt.ylim(-1, 1)
                        
                    else:
                        raise Exception("Something went really wrong.")
                        
                    plt.plot(data_x, data_y, color=COLORS[age_n], alpha=1 / RUN_COUNT * 5, label=age if age not in was else None)
                    was.add(age)

        plt.xlabel("Datum")
        plt.ylabel(" ".join(label.split()[:2]))

        # plt.legend()

        plt.title(label + f" pro {P=} a věkovou skupinu {age}")
        plt.savefig(os.path.join(current_directory, f"age={age_category}__{P=}.png"), facecolor="white", transparent=False)

        plt.clf()

/tmp/ipykernel_9841/2836786216.py:18: RuntimeWarning: divide by zero encountered in divide
  data_y = 1 - (np.array(y11[age]) / np.array(y12[age])) / (np.array(y21[age]) / np.array(y22[age]))


<Figure size 1600x900 with 0 Axes>

<Figure size 1600x900 with 0 Axes>

<Figure size 1600x900 with 0 Axes>

<Figure size 1600x900 with 0 Axes>

<Figure size 1600x900 with 0 Axes>

<Figure size 1600x900 with 0 Axes>

<Figure size 1600x900 with 0 Axes>